# Setup Libraries & Constant

In [26]:
import sys
import ast
import pandas as pd
import docplex.mp
from collections import namedtuple
from docplex.mp.model import Model
from docplex.mp.environment import Environment
import docplex.mp.conflict_refiner as cr
import random

# Define Constant

## Country & Economics

In [27]:
int_cod = ('JPN',  'LKA',  'HKG',  'CHN',  'SGP',  'MYS',  'KOR',  'IND',  'OAS',  'PAK',  'IDN',  'PHL',  'BGD',  'THA',  'VNM',  'KHM',  'MMR')
int_country = {'BD': 'BGD','CN': 'CHN','HK': 'HKG','IN': 'IND','ID': 'IDN', 'JP': 'JPN',
  'KH': 'KHM','KR': 'KOR','MM': 'MMR', 'MY': 'MYS','PK': 'PAK','PH': 'PHL','SG': 'SGP',
   'LK': 'LKA',  'TW': 'OAS',  'TH': 'THA',  'VN': 'VNM' }
con_int = dict(zip(int_country.values(),int_country.keys()))

assert len(int_cod),len(int_country)
assert len([i for i in int_country.values() if i not in int_cod]) ==0
regions_cod = ('AFR','EUR','ELT','WLT','MDE','NAM','OCE')
economics = tuple(int_cod + regions_cod)

## Vessel

In [28]:
VESSEL_TYPE = [
    ('Feeder',      3000,   8,   170, 1700, 146 ),
    ('Panamax',     8500,   11,  275, 1100, 220),
    ('Neo_Panamax', 14000,  13,  335, 600,  326),
    ('ULCV',        25000,  14,  390, 200,  506)
]
# 170 275 335 390
Vessel_type = namedtuple('vessel_type',['type','teu','draft','length','quantity','deploy_cost'])
# Cost in week base (k$)
vtypes = [Vessel_type(*v) for v in VESSEL_TYPE]
vtypes

# Note. Don't forget to adjust ship building year
# Vessel 				Size			DWT(Med)	Depth		Length (Min,Med,Max,Mean)
# Feeder 				(<3000 Teu)		20k			<=8			78,170,247,168
# Panamax  			(3000-8500 Teu)		65k			<=11		175,275,366,277
# Post / Neo Panamax 	(8500-14000 Teu)120k		<=13		298,336,400,338
# ULCV 				(>14000 Teu)		190k		<=14		353,399,400,389

[vessel_type(type='Feeder', teu=3000, draft=8, length=170, quantity=1700, deploy_cost=146),
 vessel_type(type='Panamax', teu=8500, draft=11, length=275, quantity=1100, deploy_cost=220),
 vessel_type(type='Neo_Panamax', teu=14000, draft=13, length=335, quantity=600, deploy_cost=326),
 vessel_type(type='ULCV', teu=25000, draft=14, length=390, quantity=200, deploy_cost=506)]

In [29]:
vtypes[3].teu


25000

# Import data

## Demand

In [ ]:
year = input("Year Select (2012-2015) : ")

In [30]:
dem_matrix = pd.read_csv(f'load_matrix_{year}.csv').iloc[:,1:]
# Demand = namedtuple('Frm',['teu','draft','length','quantity','deploy_cost'])
# OD = Origin Destination
demands,tot_dem, = {},{}
for f in economics:
    tot_dem[f] = 0
    for t in economics:
        if f==t : continue
        demands[f,t] = dem_matrix[(dem_matrix.From==f)&(dem_matrix.To==t)].TEU.sum()//50
        tot_dem[f] += demands[f,t]
assert len(demands) == len(economics) *(len(economics)-1)
print(f"Total Demand {len(demands)}")    #552
"('JPN', 'LKA') : ",demands[('JPN', 'LKA')],"tot_dem['JPN']",tot_dem['JPN']

Total Demand 552


("('JPN', 'LKA') : ", 180, "tot_dem['JPN']", 113678)

In [31]:
annual_teu = pd.read_csv(f'TEU_{year}.csv')
dem = dict(zip(annual_teu.Econ,annual_teu .TEU//50))

## Port

In [33]:
master_port = pd.read_csv('ports_master.csv').iloc[:,1:]
# port = master_port[['LOCODE','draft0_r','draft1_r','draft2_r','draft3_r']].itertuples(index=False)
PORTS = list(master_port[['LOCODE','draft0_r','draft1_r','draft2_r','draft3_r','due_0', 'due_1', 'due_2', 'due_3', 'thc/teu','tranship/teu']].itertuples(index=False, name=None))
# Port = namedtuple('port',['LOCODE','draft0','draft1','draft2','draft3','due_0', 'due_1', 'due_2', 'due_3', 'thc/teu','tranship/teu'])
# PORT[]
PORTS [55:61],len(PORTS)


([('PKBQM', 0, 0, 670, 780, 24831, 65081, 122581, 122581, 51, 30),
  ('PKKHI', 0, 0, 1675, 1560, 46012, 131762, 254262, 254262, 128, 0),
  ('SGSIN', 1700, 1375, 0, 17160, 4018, 5768, 8268, 8268, 65, 42),
  ('THSGZ', 510, 0, 0, 0, 5399, 15899, 30899, 30899, 47, 1),
  ('THBKK', 2890, 0, 0, 0, 5399, 15899, 30899, 30899, 47, 1),
  ('THLCH', 0, 0, 1675, 3120, 10793, 28293, 53293, 53293, 41, 32)],
 67)

In [34]:
# EX_PORT = , 0, 0, 0, 0, 23110, 35360, 52860, 52860, 0, 31)
for reg in regions_cod:
    sc = (reg, 20000, 20000, 20000, 20000, 0, 0, 0, 0, 0, 999)
    PORTS.append(sc)
# len(sc),len(ports[2]),sc
len(PORTS) #75
# High transhipment, Low THC, due ???, unlimited berth

74

In [35]:
interest_port = tuple(master_port.LOCODE.tolist())
total_port = tuple(master_port.LOCODE.tolist() + list(regions_cod))
total_port[:3],total_port[-3:],total_port.index('LKCMB'),PORTS[44]

(('BDCGP', 'CNDCB', 'CNYTG'),
 ('MDE', 'NAM', 'OCE'),
 43,
 ('MMRGN', 4760, 0, 0, 0, 36380, 104630, 202130, 202130, 52, 1))

In [36]:
Ports_info = namedtuple('ports_info',['lOCODE','draft0_r','draft1_r','draft2_r','draft3_r','due_0', 'due_1', 'due_2', 'due_3', 'thc','tranship'])
ports = [Ports_info(*p) for p in PORTS]
ports [:2],len(ports)

([ports_info(lOCODE='BDCGP', draft0_r=3060, draft1_r=0, draft2_r=0, draft3_r=0, due_0=36380, due_1=104630, due_2=202130, due_3=202130, thc=52, tranship=1),
  ports_info(lOCODE='CNDCB', draft0_r=0, draft1_r=0, draft2_r=0, draft3_r=2340, due_0=15797, due_1=24547, due_2=37047, due_3=37047, thc=89, tranship=31)],
 74)

In [38]:
# port_country [_p] == _e
port_country = {i:economics.index(int_country[p[:2]]) for i,p in enumerate(total_port) if len(p)==5}
port_country.update({i:economics.index(p) for i,p in enumerate(total_port) if len(p)==3})
# port_country += [(p,i,p) for i,p in enumerate(total_port) if len(p)==3]
port_country[2],economics[3],port_country[73],total_port[73],economics[23]

(3, 'CHN', 23, 'OCE', 'OCE')

## Port Call

In [39]:
port_call_df = pd.read_csv('Port_call_master.csv').iloc[:,1:]
port_call_df = port_call_df[port_call_df.Calls_Count>0].reset_index(drop=True)
def str2list(x:str):
    return list(ast.literal_eval(x))
port_call_df.LOCODE_Calls = port_call_df.LOCODE_Calls.apply(str2list)
port_call_df.Calls = port_call_df.Calls.apply(str2list)
port_call_df[['Calls','PC_Count','Area_Calls','Area_Cover','Turn_Around','Week','Calls_Count']].iloc[102]

Calls          [CNXMG, TWKHH, TWTXG, CNXMG, TWKHH, CNXMG]
PC_Count                                                6
Area_Calls                                            INT
Area_Cover                                              3
Turn_Around                                             7
Week                                                    1
Calls_Count                                             6
Name: 102, dtype: object

In [40]:
PORTCALL = list(port_call_df[['Calls','PC_Count','Area_Calls','Area_Cover','Turn_Around','Week','Calls_Count']].itertuples(index=False, name=None))
PORTCALL[92]
port_call_df[['LOCODE_Calls','Calls','PC_Count','Area_Calls','Area_Cover','Turn_Around','Week','Calls_Count']].iloc[92]
# port_call_df.Calls_Count.value_counts()

LOCODE_Calls    [CNNBG, CNSHG, KRPUS, USLGB, KRPUS, CNNBG]
Calls               [NAM, KRPUS, CNNBG, CNSHG, KRPUS, NAM]
PC_Count                                                 6
Area_Calls                                         INT,NAM
Area_Cover                                               7
Turn_Around                                             42
Week                                                     6
Calls_Count                                              6
Name: 92, dtype: object

In [41]:
# Store as index 
routes_ind = []
cnt,ind = 0,0
conv_bin = lambda x: 0 if x==0 else 1
for ins in PORTCALL:
    pc = ins[0]
    route = []
    legs= ins[-1]
    cnt += legs
    wk = ins[-2]
    econ = []
    for l in range(legs):
        call_port = pc[l]
        # econ.append(port_country[call])
        # print(call_port)
        try:
            call = total_port.index(call_port)
        except:
            print(pc,len(routes_ind),call_port)
            raise
        route.append(call)
        econ.append(port_country[call])
    econ = tuple(dict.fromkeys(econ))
    routes_ind.append((wk,legs,tuple(pc),tuple(route),econ))
    ind += 1

len(routes_ind),cnt,routes_ind[3][:4]
# [size,legs,pc(locode),pc(index),economics include]

(431, 3501, (1, 3, ('SGSIN', 'KHKOS', 'SGSIN'), (57, 38, 57)))

In [43]:
# Check if @ full berth occupancy can accomodate demand
max_cap = {}
ves_cap = [3000,8500,14000,25000]
ves_size = [170,275,335,390]
meter_size = [a/b for a, b in zip(ves_cap,ves_size)]
for _f,f in enumerate(economics):           # @ Economy f
    fports = [(_p,total_port[_p]) for _p,c in port_country.items() if c == _f]
    max_cap[f] = 0
    for _p,p in fports:
        port_berth = PORTS[_p][1:5]
        max_cap[f] += int(sum([a * b for a, b in zip(port_berth, meter_size)]))
    print(f"{f},\t{max_cap[f]},\t\t{dem[f]}\t\t{max_cap[f]>dem[f]}")


JPN,	1523000,		402768		True
LKA,	246500,		103700		True
HKG,	421000,		402280		True
CHN,	5140500,		3874680		True
SGP,	1172500,		634204		True
MYS,	646500,		480254		True
KOR,	1366000,		509540		True
IND,	872500,		237660		True
OAS,	627500,		289836		True
PAK,	248000,		55112		True
IDN,	405000,		240634		True
PHL,	234500,		144208		True
BGD,	54000,		40893		True
THA,	330000,		189260		True
VNM,	385000,		221791		True
KHM,	12000,		9480		True
[17.647058823529413, 30.90909090909091, 41.791044776119406, 64.1025641025641] (4760, 0, 0, 0) 16544
MMR,	84000,		16544		True
AFR,	3088995,		299269		True
EUR,	3088995,		2644787		True
ELT,	3088995,		685540		True
WLT,	3088995,		158530		True
MDE,	3088995,		779243		True
NAM,	3088995,		1181868		True
OCE,	3088995,		244728		True


## Iterator

In [44]:
# port_capacity = 
_interest_port  = len(interest_port)
_total_port     = len(total_port)
_ports          = len(ports)
_port_country   = len(port_country)

_interest_port,_total_port,_ports,_port_country,type(interest_port),type(total_port),type(ports),type(port_country)

(67, 74, 74, 74, tuple, tuple, list, dict)

In [45]:
# econ demand
_demands        = len(demands)
_int_cod        = len(int_cod)# 3 Digit
_int_country    = len(int_country)  # 2 Digit
_regions_cod    = len(regions_cod)
_economics      = len(economics)

_demands,_int_cod,_int_country,_regions_cod,_economics,type(demands),type(int_cod),type(int_country),type(regions_cod),type(economics)

(552, 17, 17, 7, 24, dict, tuple, dict, tuple, tuple)

In [46]:
# route preparing
routes_ind
_routes = len(routes_ind)

# Vessel 
_vtypes = len(vtypes)
_routes ,_vtypes,type(routes_ind),type(vtypes)

(431, 4, list, list)

In [47]:
rv =    [(r,v)        for r in range(_routes) for v in range(_vtypes)] 
rolp =  [(r,o,l,routes_ind[r][3][l])    for r in range(_routes) for o in range(_economics) 
                                        for l in range(routes_ind[r][1])]
ope =   [(o,_p,e )    for o in range(_economics) for _p,e in port_country.items()]   
pv =    [(p,v)        for p in range(_total_port) for v in range(_vtypes)]
# Obsolete po =    [(p,o)        for p in range(_total_port) for o in range(_economics)] 
#routes_ind[2]
# po[32]
len(rolp),len(ope),

(84024, 1776)

# Setup Model

In [52]:
env = Environment()
env.print_information()
print('/n')
print("#"*15)
print(f"Model {year} Initiate")
print("#"*15)
mdl = Model(f"Container_network_3.2_{year}")


* system is: Windows 64bit
* Python version 3.8.5, located at: C:\ProgramData\Anaconda3\python.exe
* docplex is present, version is 2.18.200
* CPLEX library is present, version is 20.1.0.0, located at: C:\ProgramData\Anaconda3\lib\site-packages
* pandas is present, version is 1.1.3


## Decision Variable

In [53]:
# Vessel in route (Ves=>)
ves_deploy = mdl.integer_var_dict(rv,  name="ves_deploy") # Maximum 7 day/weeks call

# # Load, Unload, and Carried
load = mdl.continuous_var_dict(rolp,name="load")
unload = mdl.continuous_var_dict(rolp, name="unload")
carried = mdl.continuous_var_dict(rolp, name="carried")

            # # print(v,o,r[0],r[])
# Import,Export
imp = mdl.continuous_var_dict(ope,name="imp")
exp = mdl.continuous_var_dict(ope,name="exp")
# trade = mdl.continuous_var_dict(ope,name="trade")

## Cal Var


In [54]:
port_calls =  mdl.continuous_var_dict(pv,name="port_calls")
route_cap =  mdl.continuous_var_list(range(_routes),name="route_cap")
port_load =  mdl.continuous_var_dict(ope,name="port_load")
port_unload = mdl.continuous_var_dict(ope,name="port_unload")
port_trans  = mdl.continuous_var_dict(ope,name="port_trans")
port_thrp =  mdl.continuous_var_list(range(_total_port),name="port_thrp")

len(port_thrp )

74

## Constraint

### CR1 Capacity of each route

In [57]:
# all o in econ all r in routes 
# sum(load - unload for l in r[l])

for r in range(_routes):
    mdl.add_constraint(
            ## deploy * size 
            mdl.sum((ves_deploy[(r,v)]*vtypes[v].teu) for v in range(_vtypes)) 
            # + relax_route_cap[r]    # Relaxed
            == route_cap[r],f"Cap route {r}"
        ) # 

### C2 Vessel Availability (Vessel deploy according to weekly service)
### C3 Port Total Call (by types)

In [58]:
for v in range(_vtypes):
    mdl.add_constraint(
            ## deploy * week
            mdl.sum((ves_deploy[(r,v)]*routes_ind[r][0]) for r in range(_routes)) 
            <= vtypes[v].quantity,f"Availability type {v}")

    # Fix this shit
    for _p,p in enumerate(total_port): 
        mdl.add_constraint(mdl.sum(ves_deploy[(r,v)]/24 
            for r,o,l,p in rolp if p==_p and l<routes_ind[r][1]-1) # Use 1 to reduce end duplicate 
            ==port_calls[(_p,v)],f"Total Call port {p}, type {v}")

### C4 Carried less than vessel capacity

In [60]:
for r in range(_routes):
    for o in range(_economics):
        mdl.add_constraint( mdl.sum((load[(r,o,l,p)] - unload[(r,o,l,p)]) 
        for l in range(routes_ind[r][1]-1) for p in [routes_ind[r][3][l]])
        == 0,
        f"Route flow conservation {r}{o}"  )

    for l in range(routes_ind[r][1]):
        p = routes_ind[r][3][l]

        mdl.add_constraint( mdl.sum(carried[(r,o,l,p)] 
            for o in range(_economics))
            <= route_cap[r],f"Cap & Carried  {r},{l}")

### C5 Port Flow (vessel) Continuity (per each berthing) 
### C6 Circular Route (PC[0] ==PC[-2])

In [63]:
for _rol in rolp :
    r,o,l,p = _rol
    # No unload at origin (no import @ origin)
    if port_country[p]==o:
        mdl.add_constraint(unload[_rol]==0,f"No origin unload {p},{o}")
        
    # first == last portcall since is identical with first portcall see C5
    if l==routes_ind[r][1]-1:    
        # print(r,l,"  x") 
        p0 = routes_ind[r][3][0]
        mdl.add_constraint(carried[_rol]== carried[(r,o,0,p0)] ,f"Circ Carried {_rol}")
        mdl.add_constraint(load[_rol]   == load[(r,o,0,p0)]   , f"Circ Load {_rol}")
        mdl.add_constraint(unload[_rol] == unload[(r,o,0,p0)]   ,f"Circ unload {_rol}")
    elif l<routes_ind[r][1]-1 :
        # print(r,l,)
        _p = routes_ind[r][3][l+1]
        mdl.add_constraint(carried[(r,o,l+1,_p)] == carried[_rol] + load[_rol] - unload[_rol],f"R Continuity {_rol}")

### C7 Port Load, Unload, No internal trade, & Throughput
TH = load + unload
For port in e can only load cargoes from e

In [64]:

for _o,_p,_e in ope:
    # Convervation of flow @ port
    mdl.add_constraint( mdl.sum((load[(r,o,l,p)] - unload[(r,o,l,p)] )
                            for r,o,l,p in rolp 
                            if p==_p and o==_o and l<routes_ind[r][1]-1) 
                        - exp[(_o,_p,_e)] + imp[(_o,_p,_e)]
                
                        == 0,
                        f"Port Flow conservation {_p},{_o},{_e}")

    # Port load
    mdl.add_constraint( mdl.sum(load[(r,o,l,p)] 
                            for r,o,l,p in rolp 
                            if p==_p and o==_o and l<routes_ind[r][1]-1) 
                        == port_load[(_o,_p,_e)],
                        f"Sum Port Load {_p},{_o},{_e}")
    # mdl.add_kpi(port_load, "Port_load")

    # Port unload
    mdl.add_constraint( mdl.sum(unload[(r,_o,l,p)]  
                        for r,o,l,p in rolp if p==_p and o==_o and l<routes_ind[r][1]-1) 
                        == port_unload[(_o,_p,_e)],
                        f"Sum Port Unload {_p},{_o},{_e}") 
    # mdl.add_kpi(port_unload, "Port_unload")
                        
    if _o == _e:
        # Port unload product 
        mdl.add_constraint( port_unload[(_o,_p,_e)] == 0,
                                f"Port Unload Fix=0 {_p},{_o}")
    else:
        mdl.add_constraint( port_unload[(_o,_p,_e)] + port_load[(_o,_p,_e)] - imp[(_o,_p,_e)]
                            ==2*port_trans[(_o,_p,_e)],
                            f"Port_trans_{p}_{_p}_{_e}")

    # Flow conservation
    # mdl.add_constraint( port_unload[(_o,_p,_e)] + ,
    #                     f"Port Unload {_p},{_o},{_e}")

for _p,p in enumerate(total_port):
    _e = port_country[_p]
    # Port Throughput
    mdl.add_constraint( mdl.sum(port_unload[(_o,_p,_e)] for _o,o in enumerate(economics))
                        + mdl.sum(port_load[(_o,_p,_e)] for _o,o in enumerate(economics))
                        ==port_thrp[_p],f"Port Throughput {p}")
    
    ## Export import conservation

    

### CR8 Berth Draft Limitation

In [65]:
for v in range(_vtypes):
    _v = _vtypes - v -1
    print(sum([PORTS[_p][1+v_] for v_ in range(_v,_vtypes)]))
    for _p,p in enumerate(total_port) :
        mdl.add_constraint(sum([PORTS[_p][1+v_]*7 for v_ in range(_v,_vtypes)]) 
                        >= mdl.sum((port_calls[(_p,v_)] * vtypes[v_].length) for v_ in range(_v,_vtypes))
                        ,f"Berth Limitation port {p} type {v}")


20000
40000
60000
80000


### C9 Trade for each country
### C10 Demand Satisfied (Only in interested Area)
### C11 No internal trade


In [66]:
## Relaxing External FLow
# imp = mdl.continuous_var_dict(ope,name="imp")
# exp = mdl.continuous_var_dict(pe,name="exp")
for _f,f in enumerate(economics):           # @ Economy f
    fports = [(_p,total_port[_p]) for _p,c in port_country.items() if c == _f]
    # tports = [(_p,total_port[_p]) for _p,c in port_country.items() if c == _t]
    if f in int_cod:
        # export from interest port
        mdl.add_constraint(mdl.sum(exp[(_e,_p,_f)] 
                            for _p,p in fports 
                            for _e,e in enumerate(economics))
                        ==tot_dem[f],
                        f"export_intreg_{f}")
    else:
        # export from ex region
        mdl.add_constraint(mdl.sum(exp[(_e,_p,_f)] 
                            for _p,p in fports 
                            for _e,e in enumerate(economics))
                        <=tot_dem[f],
                        f"export_extreg_{f}")

    for _t,t in enumerate(economics):       # Partner
        if f==t:
            # Import of (product form econ t) from econ t is 0
            mdl.add_constraints([imp[(_t,_p,_f)]==0 for _p,p in fports],
                                [f"import_none_{p} for {f}" for _p,p in fports]) 
        else:
            # Export of (product form econ t) from econ not t is 0 
            mdl.add_constraints([exp[(_t,_p,_f)]==0 for _p,p in fports],
                                [f"export_none_{p} for {f}" for _p,p in fports])
            if f in int_cod:
                # import from t to f (interested area)
                mdl.add_constraint(mdl.sum(imp[(_t,_p,_f)] 
                                for _p,p in fports)==demands[t,f],
                                f"demands_satisfy_{t,f}")
            else:
                # import from t to f (outside area)
                mdl.add_constraint(mdl.sum(imp[(_t,_p,_f)] 
                                for _p,p in fports)<=demands[t,f],
                                f"demands_relaxed_{t,f}")

## Cost Calculation

In [67]:
# Vessel deploy cost
# ** Assign cost for external 
ves_deploy_cost = mdl.sum(ves_deploy[(r,v)] * vtypes[v].deploy_cost for r,v in rv)
# Port Call cost
port_call_cost = mdl.sum(ves_deploy[(r,v)] * ports[_p][v+5] for r,v in rv
                            for l in range(routes_ind[r][1]-1) 
                            for p in [routes_ind[r][3][l]])
# Load & unload cost
# ** Adjust cost for external 
handling_cost = mdl.sum(port_thrp[_p] * ports[_p][-2] for _p in range(_total_port))

# Transhipment cost
trans_cost = mdl.sum(port_trans[(o,_p,e)] * ports[_p][-1] for o,_p,e in ope)


## RUN

In [70]:
# mdl.maximize(handling_cost + ves_deploy_cost + trans_cost)
mdl.minimize(ves_deploy_cost+handling_cost + ves_deploy_cost +trans_cost)
mdl.print_information()


Model: Container_network_2
 - number of variables: 264208
   - binary=0, integer=2455, continuous=261753
 - number of constraints: 130913
   - linear=130913
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [71]:
s = mdl.solve(log_output=True)
if s is not None:
    mdl.report()
else:
    print("* model is infeasible")

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              202001241
Tried aggregator 3 times.
MIP Presolve eliminated 31847 rows and 42830 columns.
MIP Presolve modified 159 coefficients.
Aggregator did 22546 substitutions.
Reduced MIP has 75747 rows, 198832 columns, and 617171 nonzeros.
Reduced MIP has 394 binaries, 953 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.67 sec. (502.88 ticks)
Probing fixed 0 vars, tightened 2 bounds.
Probing time = 0.09 sec. (14.55 ticks)
Cover probing fixed 0 vars, tightened 12 bounds.
Tried aggregator 2 times.
Detecting symmetries...
MIP Presolve eliminated 97 rows and 280 columns.
MIP Presolve modified 260 coefficients.
Aggregator did 18 substitutions.
Reduced MIP has 75632 rows, 198534 columns, and 616302 nonzeros.
Reduced MIP has 422 binaries, 1050 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.50 sec. (342.01 ticks)
Probing time = 0.06 sec. 

In [ ]:
mdl.print_solution()

In [ ]:
total_dvar = len(rolp)*3 + len(ope)*2 + len(rv)
print(f"Total Dvar {total_dvar}")

In [74]:
import time
tref = time.strftime("%m%d%H%M".time.localtime())
mdl.export_as_lp(path='.')
mdl.solution.export(f"Solution_{year}_{tref}.json")
assert 1 == 2


objective: 15967247213.000
  ves_deploy_7_3=2
  ves_deploy_10_0=1
  ves_deploy_14_2=1
  ves_deploy_18_1=1
  ves_deploy_25_0=1
  ves_deploy_25_1=1
  ves_deploy_29_2=1
  ves_deploy_34_1=1
  ves_deploy_34_3=1
  ves_deploy_41_3=1
  ves_deploy_42_0=2
  ves_deploy_49_3=1
  ves_deploy_61_2=1
  ves_deploy_65_3=3
  ves_deploy_69_2=1
  ves_deploy_71_3=1
  ves_deploy_75_0=1
  ves_deploy_96_0=1
  ves_deploy_96_1=1
  ves_deploy_100_1=1
  ves_deploy_111_3=1
  ves_deploy_112_3=1
  ves_deploy_113_0=1
  ves_deploy_121_3=1
  ves_deploy_129_0=1
  ves_deploy_129_3=1
  ves_deploy_131_0=1
  ves_deploy_142_0=1
  ves_deploy_146_0=3
  ves_deploy_146_3=1
  ves_deploy_163_0=1
  ves_deploy_179_1=1
  ves_deploy_229_2=1
  ves_deploy_427_0=1
  ves_deploy_427_3=1
  load_7_3_1_57=1697.000
  load_7_4_1_57=5529.000
  load_7_7_1_57=121.000
  load_7_12_1_57=151.000
  load_7_13_1_57=4245.000
  load_7_14_1_57=1906.000
  load_7_16_1_57=7.000
  load_7_23_0_73=50000.000
  load_7_23_2_73=50000.000
  load_10_3_1_7=3000.000
  loa

AssertionError: 

## Refiner

In [ ]:
cref = cr.ConflictRefiner()
print('show some of the constraints that can be removed to arrive at a minimal conflict')
refine = cref.refine_conflict(mdl, display=True)
# refine.diplay()
type(refine)
conflicts = refine.iter_conflicts()
refine.display_stats()

show some of the constraints that can be removed to arrive at a minimal conflict


In [ ]:
# for c in conflicts:
#     print(c)

## RELAXER

In [ ]:
from docplex.mp.relax_linear import LinearRelaxer
bmr = LinearRelaxer.make_relaxed_model(mdl)
bmr.print_information()
rs = bmr.solve(log_output=True)
assert rs

Model: lp_Container_network_2
 - number of variables: 263477
   - binary=0, integer=0, continuous=263477
 - number of constraints: 120569
   - linear=120569
 - parameters: defaults
 - objective: minimize
 - problem type is: LP
Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              202001241
Parallel mode: deterministic, using up to 8 threads for concurrent optimization:
 * Starting dual Simplex on 1 thread...
 * Starting Barrier on 6 threads...
 * Starting primal Simplex on 1 thread...
Tried aggregator 1 time.
LP Presolve eliminated 30636 rows and 103824 columns.
Aggregator did 84231 substitutions.
Reduced LP has 5702 rows, 75422 columns, and 235902 nonzeros.
Presolve time = 0.53 sec. (314.02 ticks)

Iteration log . . .
Iteration:     1   Dual objective     =             0.000000
Perturbation started.
Iteration:   101   Dual objective     =             0.000000
Iteration:   411   Dual obj

AssertionError: 

In [ ]:
type(rs)

NoneType

In [ ]:
rs.display()

AttributeError: 'NoneType' object has no attribute 'display'

# Model Develop

In [ ]:
port_dem = {}
for _f,f in enumerate(economics):           # @ Economy f
    fports = [(_p,total_port[_p]) for _p,c in port_country.items() if c == _f]
    print(f,_f,len(fports))

JPN 0 9
LKA 1 1
HKG 2 1
CHN 3 14
SGP 4 1
MYS 5 5
KOR 6 4
IND 7 11
OAS 8 4
PAK 9 2
IDN 10 3
PHL 11 5
BGD 12 1
THA 13 3
VNM 14 2
KHM 15 1
MMR 16 1
AFR 17 1
EUR 18 1
ELT 19 1
WLT 20 1
MDE 21 1
NAM 22 1
OCE 23 1


### CX Port Cap ?

In [ ]:
port_country
ope = [(p[:2],p,e)for p in interest_port for e in economics]
len(ope),_ports*_economics
        


(1632, 1632)

## Model Residue

In [ ]:
# Load from port to route
cnt = 0
for r in range(_routes):
    for o in range(_economics):
        for l in range(routes_ind[r][1]):
            cnt +=1
# Unload from route to port
unload = mdl.integer_var_matrix( keys1=matrix_region,keys2=products, ub=0.8, name="price_prod_econ")
# Carried to next leg
carried = mdl.integer_var_matrix( keys1=matrix_region,keys2=products, ub=0.8, name="price_prod_econ")

x = m.integer_var_dict((f, r, r['Leg'], d)
                       for f in economics
                       for r in routes
                       for d in r['Call'])

                       


NameError: name 'matrix_region' is not defined

In [ ]:
mdl.export_as_lp()

'C:\\Users\\AN515-52\\AppData\\Local\\Temp\\Container_network_2.lp'

In [ ]:
len(economics),len(routes),3522/435

(24, 435, 8.096551724137932)

In [ ]:
vtypes = (0,1,2,3)
cnt = 0
sel = int(random.random() * len(routes) //1)    # 3522 total leg
print(sel)
for r in routes:
    for o in economics:
        for l in range(r[1]):
            # print(v,o,r[0],r[])
            cnt +=1
cnt
# 84528 = 3522 * 

366


84528

In [ ]:
routes = [{'Name':'A','Leg':5,'Call':['X','Y','Z']},{'Name':'B','Leg':7,'Call':['X','Y','X']},{'Name':'A','Leg':9,'Call':['Z','Y','Z']}]
# [(i,j,p) for i in routes for j in i for p in i['Call']]

In [ ]:
# Internal Teu Allocation (Economics to port
for c in matrix_region for e in matrix_region for 

In [ ]:
# https://stackoverflow.com/questions/55821484/how-to-set-a-four-dimension-variable-in-docplex-with-python 
x = m.binary_var_dict((i, l, t, v, r)
                       for i in types
                       for l in locations
                       for t in times
                       for v in vehicles
                       for r in routes)

for i in types:
    for l in locations:
       for t in times:
          for v in vehicles:
             for r in routes:
                print x[i, l, t, v, r]

x = [[[[[m.binary_var('%d_%d_%d_%d_%d' % (i, l, t, v, r))
        for r in routes]
        for v in vehicles]
    for t in times]
    for l in locations]
    for i in types]

for i in types:
for l in locations:
    for t in times:
        for v in vehicles:
            for r in routes:
            print x[i][l][t][v][r]

In [ ]:
from docplex.mp.model import Model

# Data

r=range(1,3)

i=[(a,b,c,d) for a in r for b in r for c in r for d in r]

print(i)

mdl = Model(name='model')

#decision variables
mdl.x=mdl.integer_var_dict(i,name="x")

# Constraint
for it in i:
    mdl.add_constraint(mdl.x[it] == it[0]+it[1]+it[2]+it[3], 'ct')

mdl.solve()

# Dislay solution
for it in i:
    print(" x ",it," --> ",mdl.x[it].solution_value); 

# Model route travel
Delete INMRM

In [ ]:
env = Environment()
env.print_information()
mdl2 = Model("Route_Mockup_2")
ves_deploy = mdl2.integer_var_dict(rv, ub=3, name="ves_deploy") # Maximum 7 day/weeks call
port_calls =  mdl2.integer_var_dict(pv,name="port_calls")
route_cap =  mdl2.integer_var_list(range(_routes),name="route_cap")
calls_per_port =  mdl2.integer_var_list(range(_total_port),name="port_calls")

* system is: Windows 64bit
* Python version 3.8.5, located at: C:\ProgramData\Anaconda3\python.exe
* docplex is present, version is 2.18.200
* CPLEX library is present, version is 20.1.0.0, located at: C:\ProgramData\Anaconda3\lib\site-packages
* pandas is present, version is 1.1.3


In [ ]:
for r in range(_routes):
    # C1 Capacity of each route
    mdl2.add_constraint(
            ## deploy * size 
            mdl2.sum((ves_deploy[(r,v)]*vtypes[v].teu) for v in range(_vtypes)) 
            == route_cap[r],f"Cap route {r}"
        ) 
for v in range(_vtypes):
    # C2 Vessel Availability (Vessel deploy according to weekly service)
    mdl2.add_constraint(
            ## deploy * size 
            mdl2.sum((ves_deploy[(r,v)]*routes_ind[r][0]) for r in range(_routes)) 
            <= vtypes[v].quantity,f"Availability type {v}"
        )
    # C3 Port Total Call (by types)
    for _p,p in enumerate(total_port): 
        mdl2.add_constraint(mdl2.sum(ves_deploy[(r,v)] 
            for r in range(_routes) if p in routes_ind[r][2] ) # Use 1 to reduce end duplicate 
            ==port_calls[(_p,v)]),f"Total Call port {p}, type {v}"


In [ ]:
[r for r in range(_routes) if 'INMRM' in routes_ind[r][2]]

[]

In [ ]:

for _p,p in enumerate(total_port):
    # C4 At least 1 call per port
    mdl2.add_constraint(mdl2.sum(port_calls[(_p,v)] for v in range(_vtypes)) >=1 )
    mdl2.add_constraint(mdl2.sum(port_calls[(_p,v)] for v in range(_vtypes)) == calls_per_port[_p])

In [ ]:
ves_deploy_cost=    mdl2.sum(ves_deploy[(r,v)] * vtypes[v].deploy_cost for r,v in rv)
mdl2.maximize(ves_deploy_cost)
mdl2.print_information()
s = mdl2.solve(log_output=True)
mdl2.export_as_lp(path='.')
if s is not None:
    mdl2.report()
else:
    print("* model is infeasible")

Model: Route_Mockup_2
 - number of variables: 2525
   - binary=0, integer=2525, continuous=0
 - number of constraints: 879
   - linear=879
 - parameters: defaults
 - objective: maximize
 - problem type is: MILP
Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              202001241
Tried aggregator 2 times.
MIP Presolve eliminated 74 rows and 104 columns.
Aggregator did 727 substitutions.
Reduced MIP has 78 rows, 1694 columns, and 11562 nonzeros.
Reduced MIP has 0 binaries, 1694 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.05 sec. (8.32 ticks)
Found incumbent of value 10546.000000 after 0.16 sec. (50.60 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 78 rows, 1694 columns, and 11562 nonzeros.
Reduced MIP has 0 binaries, 1694 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (3.53 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search meth

In [ ]:
mdl2.solution.get_value_list(calls_per_port)

[60.0,
 6.0,
 12.0,
 42.0,
 27.0,
 33.0,
 114.0,
 111.0,
 81.0,
 183.0,
 264.0,
 180.0,
 67.0,
 214.0,
 313.0,
 400.0,
 42.0,
 72.0,
 109.0,
 24.0,
 48.0,
 3.0,
 21.0,
 21.0,
 9.0,
 36.0,
 9.0,
 57.0,
 48.0,
 42.0,
 57.0,
 48.0,
 42.0,
 78.0,
 165.0,
 144.0,
 165.0,
 168.0,
 42.0,
 111.0,
 15.0,
 93.0,
 222.0,
 81.0,
 33.0,
 33.0,
 93.0,
 54.0,
 218.0,
 307.0,
 24.0,
 48.0,
 42.0,
 27.0,
 117.0,
 33.0,
 18.0,
 580.0,
 24.0,
 75.0,
 255.0,
 135.0,
 69.0,
 123.0,
 350.0,
 152.0,
 264.0,
 15.0,
 1.0,
 1.0,
 1.0,
 75.0,
 22.0,
 21.0]

In [ ]:
price = mdl2.solution.get_value_dict(port_calls)
price

{(0, 0): 24.0,
 (0, 1): 21.0,
 (0, 2): 12.0,
 (0, 3): 3.0,
 (1, 0): 3.0,
 (1, 1): 3.0,
 (1, 2): 0,
 (1, 3): 0,
 (2, 0): 3.0,
 (2, 1): 3.0,
 (2, 2): 3.0,
 (2, 3): 3.0,
 (3, 0): 18.0,
 (3, 1): 18.0,
 (3, 2): 6.0,
 (3, 3): 0,
 (4, 0): 9.0,
 (4, 1): 9.0,
 (4, 2): 6.0,
 (4, 3): 3.0,
 (5, 0): 12.0,
 (5, 1): 12.0,
 (5, 2): 6.0,
 (5, 3): 3.0,
 (6, 0): 45.0,
 (6, 1): 33.0,
 (6, 2): 24.0,
 (6, 3): 12.0,
 (7, 0): 51.0,
 (7, 1): 33.0,
 (7, 2): 18.0,
 (7, 3): 9.0,
 (8, 0): 42.0,
 (8, 1): 27.0,
 (8, 2): 12.0,
 (8, 3): 0,
 (9, 0): 90.0,
 (9, 1): 63.0,
 (9, 2): 24.0,
 (9, 3): 6.0,
 (10, 0): 135.0,
 (10, 1): 81.0,
 (10, 2): 42.0,
 (10, 3): 6.0,
 (11, 0): 90.0,
 (11, 1): 60.0,
 (11, 2): 27.0,
 (11, 3): 3.0,
 (12, 0): 31.0,
 (12, 1): 24.0,
 (12, 2): 9.0,
 (12, 3): 3.0,
 (13, 0): 103.0,
 (13, 1): 72.0,
 (13, 2): 27.0,
 (13, 3): 12.0,
 (14, 0): 145.0,
 (14, 1): 99.0,
 (14, 2): 51.0,
 (14, 3): 18.0,
 (15, 0): 199.0,
 (15, 1): 123.0,
 (15, 2): 66.0,
 (15, 3): 12.0,
 (16, 0): 15.0,
 (16, 1): 12.0,
 (16, 2): 1

# Model assign exp imp

# MISC

In [ ]:
def demand_model1(Trade_demand,R_TEU,cont_conv = containerized):
    products = tuple([c for c in cont_conv.keys() if cont_conv[c]!=0 ])
    global int_cod,matrix_region
    mdl = Model(name='trade_to_teu_1')
    
    price_prod_econ = mdl.continuous_var_matrix( keys1=matrix_region,keys2=products, ub=0.8, name="price_prod_econ") 
    # (TEU/k$)  : Less Value => Expensive Cargo
    load_from_to =  mdl.continuous_var_matrix(keys1=matrix_region, keys2=matrix_region, name="load_from_to")
    load_total =  mdl.continuous_var_dict(keys=matrix_region, lb=10, ub = R_TEU, name="load_total")

    for f in matrix_region:
        for t in matrix_region:
            if f==t :
                continue
            mdl.add_constraint(
                mdl.sum((price_prod_econ[f,p] * Trade_demand[f,t,p]* cont_conv[p] ) for p in products) 
                == load_from_to[f,t]
            ) 
    for f in matrix_region:
        mdl.add_constraint(
            mdl.sum(load_from_to[f,t] for t in matrix_region if t!=f ) + mdl.sum(load_from_to[k,f] for k in matrix_region if k!=f)
            == load_total[f] 
            ) 
    
    total_err = mdl.sum((R_TEU[f] - load_total[f])**2 for f in int_cod)
    mdl.minimize(total_err)
    
    sol = mdl.solve()
    if sol is not None:
        # mdl.print_information()
        mdl.report()
        price = mdl.solution.get_value_dict(price_prod_econ)
        price_df = pd.DataFrame([{'Econ':k[0],'Product':k[1],'Price':v} for k,v in price.items()],columns = ['Econ','Product','Price'])
        mid = price_df.groupby('Product').Price.describe()[['50%', 'mean','min','max']]
        mid['Min_mid'] = mid[['50%', 'mean']].min(axis=1)
        mid['Max_mid'] = mid[['50%', 'mean']].max(axis=1)
        mid['Max_Uratio'] = mid['max']/mid['Max_mid'] 
        mid['Max_Lratio'] = mid['min']/mid['Max_mid'] 
        obj = sol.get_objective_value()
        # print(f"solution for a cost of {mdl.objective_value}")
        # print_information(model)
        # print_solution(model)
        return mid,obj
    else:
        print("* model is infeasible")
        return None


def demand_model2(base_price,Trade_demand,R_TEU,cont_conv = containerized):
    products = tuple([c for c in cont_conv.keys() if cont_conv[c]!=0 ])
    global int_cod,matrix_region
    mdl = Model(name='trade_to_teu_2')

    load_from_to =  mdl.continuous_var_matrix(keys1=matrix_region, keys2=matrix_region, name="load_from_to")
    load_total =  mdl.continuous_var_dict(keys=matrix_region, lb=10,ub = R_TEU, name="load_total")
    range_prod_econ = mdl.continuous_var_matrix(keys1=matrix_region, keys2=products,lb=0.002,ub=5.0, name="range_prod_econ")

    for f in matrix_region:
        for t in matrix_region:
            if f==t :
                continue
            mdl.add_constraint(
                mdl.sum((base_price[p]* range_prod_econ[f,p] * Trade_demand[f,t,p]* cont_conv[p] ) for p in products) 
                == load_from_to[f,t]
            ) 
    for f in matrix_region:
        mdl.add_constraint(
            mdl.sum(load_from_to[f,t] for t in matrix_region if t!=f ) + mdl.sum(load_from_to[k,f] for k in matrix_region if k!=f)
            == load_total[f] 
            ) 
    
    total_err = mdl.sum((R_TEU[f] - load_total[f])**2 for f in int_cod)
    mdl.minimize(total_err)
    
    sol = mdl.solve()
    if sol is not None:
        # mdl.print_information()
        mdl.report()
        load_matrix = mdl.solution.get_value_dict(load_from_to)
        C_TEU = mdl.solution.get_value_dict(load_total)
        price_range = mdl.solution.get_value_dict(range_prod_econ)
        price_range_df = pd.DataFrame([{'Econ':k[0],'Product':k[1],'Price':v} for k,v in price_range.items()],columns = ['Econ','Product','Price'])
        # print(sol.get_objective_value())
        obj = sol.get_objective_value()
        # print(f"solution for a cost of {mdl.objective_value}")
        # print_information(model)
        # print_solution(model)
        return load_matrix,C_TEU,price_range_df,obj
    else:
        print("* model is infeasible")
        return None


In [ ]:
## Relaxing External FLow
for _e,e in enumerate(economics):           # @ Economy f
    fports = [(_p,total_port[_p]) for _p,c in port_country.items() if c == _e]
    for _t,t in enumerate(economics):       # Partner
        if (f in regions_cod and t in regions_cod) :
            # External frame relaxed 
            continue
        if (f==t):
        ## Fixed this
         
            # Import = 0
            # mdl.add_constraints([imp[(_t,_p,_f)] == 0 for _p,p in fports],
            # [f"import fix {p}" for _p,p in fports])
            
            # Sum Export each port
            #### Fix this #####
            mdl.add_constraint(
                mdl.sum(exp[(_t,_p,_f)] 
                    for _p,p in fports)
                == demands[f,t] ,
                f"Sum Exp {t}"
            )

            # Flow conservation : Load == Export (No transhipment)
            mdl.add_constraints_((port_load[(_f,_p,_t)])
                >= exp[(_t,_p,_f)] for _p,p in fports),
                [f"Flow con {t},{p},{f}" for _p,p in fports]
            )
           
            
        else:    # Import Case
            # Export = 0
            # mdl.add_constraints((imp[(_t,_p,_f)] == 0 for _p,p in fports),
            # [f"export fix {p}" for _p,p in fports])
            
            # Sum Import each port
            mdl.add_constraint(
                mdl.sum(imp[(_t,_p,_f)] 
                    for _p,p in fports)
                == demands[t,f],
                f"Sum Imp {t}"
            )

            # Flow conservation : Net unload == import
            mdl.add_constraints(
                (port_unload[(_t,_p,_f)] - port_load[(_t,_p,_f)]
                >=imp[(_t,_p,_f)] for _p,p in fports),
                [f"Flow con {t},{p},{f}" for _p,p in fports]
            )
## Relaxing External FLow
for _e,e in enumerate(economics):           # @ Economy f
    fports = [(_p,total_port[_p]) for _p,c in port_country.items() if c == _e]
    for _t,t in enumerate(economics):       # Partner
        if (f in regions_cod and t in regions_cod) :
            # External frame relaxed 
            continue
        if (f==t):
        ## Fixed this
         
            # Import = 0
            # mdl.add_constraints([imp[(_t,_p,_f)] == 0 for _p,p in fports],
            # [f"import fix {p}" for _p,p in fports])
            
            # Sum Export each port
            #### Fix this #####
            mdl.add_constraint(
                mdl.sum(exp[(_t,_p,_f)] 
                    for _p,p in fports)
                == demands[f,t] ,
                f"Sum Exp {t}"
            )

            # Flow conservation : Load == Export (No transhipment)
            mdl.add_constraints_((port_load[(_f,_p,_t)]
                >= exp[(_t,_p,_f)] for _p,p in fports),
                [f"Flow con {t},{p},{f}" for _p,p in fports]
            )
           
            
        else:    # Import Case
            # Export = 0
            # mdl.add_constraints((imp[(_t,_p,_f)] == 0 for _p,p in fports),
            # [f"export fix {p}" for _p,p in fports])
            
            # Sum Import each port
            mdl.add_constraint(
                mdl.sum(imp[(_t,_p,_f)] 
                    for _p,p in fports)
                == demands[t,f],
                f"Sum Imp {t}"
            )

            # Flow conservation : Net unload == import
            mdl.add_constraints(
                (port_unload[(_t,_p,_f)] - port_load[(_t,_p,_f)]
                >=imp[(_t,_p,_f)] for _p,p in fports),
                [f"Flow con {t},{p},{f}" for _p,p in fports]
            )




# Misc

In [ ]:
# master_port.to_csv('ports_master.csv')
# master_port = sel_port.merge(port_draft,on='LOCODE')
master_port = pd.read_csv('ports_master.csv').iloc[:,1:]
master_port

,LOCODE,Country,Flow_in,Max_TEU,Min_TEU,UNLOCODE,Port_Name,draft0,draft1,draft2,draft3,Sum_len,draft0_r,draft1_r,draft2_r,draft3_r,Sum_len_r
0,BDCGP,BD,1146693.0,2806,431,BDCGP,CHATTOGRAM,3124.0,0,0,0,3124.0,3060,0,0,0,3060
1,CNDCB,CN,208187.0,6931,2770,CNDCB,DA CHAN BAY,0.0,0,0,2430,2430.0,0,0,0,2340,2340
2,CNYTG,CN,217905.0,4253,542,CNYTG,YANTAI PT,237.0,0,1492,2876,4605.0,170,0,1340,2730,4240
3,CNQZH,CN,251114.0,6096,556,CNQZH,QINZHOU PT,0.0,0,0,1533,1533.0,0,0,0,1560,1560
4,CNFOC,CN,426504.0,16022,368,CNFOC,Fuzhou,300.0,1699,0,2034,4033.0,340,1650,0,1950,3940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,TWTPE,TW,1718336.0,20190,610,TWTPE,TAIPEI,0.0,0,0,1377,1377.0,0,0,0,1560,1560
64,TWKEL,TW,1870217.0,10933,450,TWKEL,KEELUNG (CHILUNG),3349.0,660,210,403,4622.0,3400,550,335,390,4675
65,TWKHH,TW,12887313.0,23964,180,TWKHH,KAOHSIUNG,1259.0,0,2878,2565,6702.0,1190,0,3015,2730,6935
66,VNHPH,VN,1963571.0,12726,357,VNHPH,HAIPHONG,7084.0,450,0,750,8284.0,7140,550,0,780,8470


In [ ]:
master_port[master_port.Country =='MY']

,LOCODE,Country,Flow_in,Max_TEU,Min_TEU,UNLOCODE,Port_Name,draft0,draft1,draft2,draft3,Sum_len,draft0_r,draft1_r,draft2_r,draft3_r,Sum_len_r
46,MYBTU,MY,211721.0,2806,144,MYBTU,"BINTULU, SARAWAK",514.0,0,450,0,964.0,510,0,335,0,845
47,MYJHB,MY,642648.0,9404,216,MYJHB,JOHOR BAHRU,0.0,708,0,0,708.0,0,825,0,0,825
48,MYPEN,MY,606905.0,9404,653,MYPEN,PENANG (GEORGETOWN),1245.0,600,0,0,1845.0,1190,550,0,0,1740
49,MYPKG,MY,13626729.0,23112,144,MYPKG,PORT KLANG (PELABUHAN KLANG),94.0,1137,2388,1697,5316.0,170,1100,2345,1560,5175
50,MYTPP,MY,10256935.0,23656,1005,MYTPP,TANJUNG PELEPAS,0.0,0,0,5040,5040.0,0,0,0,5070,5070


In [ ]:
# Round berth lengh
def berth_round(ber,ves):
    return ves*round(ber/ves)
#  170 275 335 390
master_port['draft0_r'] = master_port.draft0.apply(lambda x: berth_round(x,170))
master_port['draft1_r'] = master_port.draft1.apply(lambda x: berth_round(x,275))
master_port['draft2_r'] = master_port.draft2.apply(lambda x: berth_round(x,335))
master_port['draft3_r'] = master_port.draft3.apply(lambda x: berth_round(x,390))
# Cumulative (Large berth can accomodate smaller vessel)
# master_port['draft3_rc'] = master_port.draft3_r
# master_port['draft2_rc'] = master_port.draft2_r +  master_port.draft3_rc
# master_port['draft1_rc'] = master_port.draft1_r +  master_port.draft2_rc
# master_port['draft0_rc'] = master_port.draft0_r +  master_port.draft1_rc


In [ ]:
# Remove selected port 
# list(zip(master_port.LOCODE,master_port.draft0 + master_port.draft1 + master_port.draft2 + master_port.draft3))
master_port['Sum_len_r'] = master_port.draft0_r + master_port.draft1_r + master_port.draft2_r + master_port.draft3_r
master_port['Sum_len_rc'] = master_port.draft0_r + master_port.draft1_r + master_port.draft2_r + master_port.draft3_r
master_port[['LOCODE','Sum_len','Sum_len_r','Sum_len_rc']].sort_values(by='Sum_len',ascending=False).head(14)

,LOCODE,Sum_len,Sum_len_r,Sum_len_rc
58,SGSIN,20239.0,20235,20235
9,CNSHG,16391.0,16130,16130
42,KRPUS,14515.0,14250,14250
6,CNNSA,12939.0,12985,12985
7,CNQDG,10201.0,10145,10145
5,CNNBG,9500.0,9360,9360
15,HKHKG,8980.0,9135,9135
12,CNXMG,8620.0,8590,8590
67,VNSGN,8376.0,8220,8220
16,IDJKT,8335.0,8530,8530


In [ ]:
master_port.to_csv('ports_master.csv')
int_country 

In [ ]:
# Fill in real througput for topport

cnt = 0
# OPE : original of cargoes, port, economics of port
# PO : port, original of cargoes
for _f,f in enumerate(economics):
    fports = [(_p,p) for c,_p,p in port_country if c == f]
    # print(f,fports,len(fports))
    for _t,t in enumerate(economics):
        if (f==t) or (f in regions_cod and t in regions_cod):
            cnt += 1
            continue
        ## Each port in f
        for _p,p in fports:
            mdl.add_constraint(imp[(_t,_p,f)]<= 
                mdl.sum(port_unload[(p_,o)] for p_,o in po if p == p_ and o!=_f),
                "Import @ {p},{f} for {t} product")         # Import from t to f
            mdl.add_constraint(exp[(_t,_p,f)]<= 
                mdl.sum(port_load[(p_,o)] for p_,o in po if p == p_ and o==_f),
                f"Export @ {p},{f} for {t} product")        # Export from f to t

        if f in int_cod:
        ## Sum every port in f (demand satisfied)
            mdl.add_constraint(mdl.sum(imp[(o,_p,e)] for o,_p,e in ope if o==_t and e == f)
                                == demands[(t,f)],
                                f"Demand (in) satisfy for {f},{t}")
            mdl.add_constraint(mdl.sum(exp[(o,_p,e)] for o,_p,e in ope if o==_f and e == f)
                                == demands[(f,t)],
                                f"Demand (in) satisfy for {f},{t}")
        else:
            ## Sum every port in f (demand relaxed satisfied)
            mdl.add_constraint(mdl.sum(imp[(o,_p,e)] for o,_p,e in ope if o==_t and e == f)
                                <= demands[(t,f)],
                                f"Demand (in) satisfy for {f},{t}")
            mdl.add_constraint(mdl.sum(exp[(o,_p,e)] for o,_p,e in ope if o==_f and e == f)
                                <= demands[(f,t)],
                                f"Demand (in) satisfy for {f},{t}")

In [ ]:
country = pd.read_csv('..//country.csv',na_filter = False).iloc[:,1:]
# country


In [ ]:
cou_reg = pd.Series(country.Economics.values,index=country.LOCODE).dropna().to_dict()
cou_reg[''] = ''
cou_reg['TW'] = 'INT'
econ_zone = dict(zip(country.LOCODE,country.Economics))
len((dict.fromkeys(econ_zone.values())))
# country[['Country_Name','Country_ISO3','Country_Code','Economics']]
econ_zone['TW'] = 'OAS'